#*```MOUNT DRIVE```*

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#*```IMPORT LIBRARIES```*

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_densenet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19

#*```DEFINE CLASS NAMES```*

In [36]:
# CELL 3: DEFINE CLASS NAMES
CLASS_NAMES = ['Immature', 'Mature', 'Normal']


#*```DEFINE MODEL PATHS AND IMAGE PATH```*

In [37]:
model_paths = {
    'densenet121': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/densenet121(99_03).h5',
    'efficientnetb3': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/efficientnetb3(99_84).h5',
    'resnet': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/resnet(99_91).h5',
    'vgg19': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/vgg19(99_89).h5'
}
confidence_dict = {}
predictions = {}
acc_dic = {'densenet121': 99.03, 'efficientnetb3': 99.84, 'resnet': 99.91, 'vgg19': 99.89}

img_path = input("image path:")

image path:/content/drive/MyDrive/PROJECT/PROJECT-1/Input Images/testcase2(n).jpg


#*```MODEL:DENSENET121 PREDICTION```*

In [38]:
print("Running DenseNet121...")
IMG_SIZE = (224, 224)
model = tf.keras.models.load_model(model_paths['densenet121'], compile=False)
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = preprocess_densenet(x)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)[0]
predictions['densenet121'] = preds
idx = np.argmax(preds)
label = CLASS_NAMES[idx]
confidence = float(preds[idx])
confidence_dict['densenet121'] = confidence
print(f"DenseNet121 Prediction: {label} (Confidence: {confidence:.2%})")

Running DenseNet121...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
DenseNet121 Prediction: Normal (Confidence: 99.90%)


#*```MODEL:EFFICIENTNETB3 PREDICTION```*

In [39]:
print("Running EfficientNetB3...")
IMG_SIZE = (224, 224)
try:
    model = tf.keras.models.load_model(model_paths['efficientnetb3'], compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    x = image.img_to_array(img)
    x = preprocess_efficientnet(x)
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)[0]
    predictions['efficientnetb3'] = preds
    idx = np.argmax(preds)
    label = CLASS_NAMES[idx]
    confidence = float(preds[idx])
    confidence_dict['efficientnetb3'] = confidence
    print(f"EfficientNetB3 Prediction: {label} (Confidence: {confidence:.2%})")
except Exception as e:
    print("Error loading or running EfficientNetB3:", e)

Running EfficientNetB3...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
EfficientNetB3 Prediction: Normal (Confidence: 100.00%)


#*```MODEL:RESNET PREDICTION```*

In [40]:
print("Running ResNet...")
IMG_SIZE = (224, 224)
model = tf.keras.models.load_model(model_paths['resnet'], compile=False)
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = preprocess_resnet(x)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)[0]
predictions['resnet'] = preds
idx = np.argmax(preds)
label = CLASS_NAMES[idx]
confidence = float(preds[idx])
confidence_dict['resnet'] = confidence
print(f"ResNet Prediction: {label} (Confidence: {confidence:.2%})")

Running ResNet...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
ResNet Prediction: Normal (Confidence: 100.00%)


#*```MODEL:VGG19 PREDICTION```*

In [41]:
print("Running VGG19...")
IMG_SIZE = (224, 224)
model = tf.keras.models.load_model(model_paths['vgg19'], compile=False)
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = preprocess_vgg19(x)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)[0]
predictions['vgg19'] = preds
idx = np.argmax(preds)
label = CLASS_NAMES[idx]
confidence = float(preds[idx])
confidence_dict['vgg19'] = confidence
print(f"VGG19 Prediction: {label} (Confidence: {confidence:.2%})")

Running VGG19...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 874ms/step
VGG19 Prediction: Normal (Confidence: 100.00%)


#*```DISPLAY INDIVIDUAL MODEL PREDICTIONS```*

In [42]:
print("\n" + "="*70)
print("SUMMARY: Individual Model Predictions")
print("="*70)
for model_name, conf in confidence_dict.items():
    pred_class = CLASS_NAMES[np.argmax(predictions[model_name])]
    print(f"{model_name:20s}: {pred_class:10s} - {conf:.4f} ({conf:.2%})")
print("="*70)
print(f"\nTotal models run: {len(confidence_dict)}")


SUMMARY: Individual Model Predictions
densenet121         : Normal     - 0.9990 (99.90%)
efficientnetb3      : Normal     - 1.0000 (100.00%)
resnet              : Normal     - 1.0000 (100.00%)
vgg19               : Normal     - 1.0000 (100.00%)

Total models run: 4


#*```CONFIDENCE MATRIX AND FINAL ENSEMBLE PREDICTION```*

In [43]:
print("\n\n" + "="*70)
print("CONFIDENCE MATRIX (Class × Model)")
print("="*70)
confidence_matrix = []
for class_name in CLASS_NAMES:
    row = []
    for model_name in model_paths.keys():
        class_idx = CLASS_NAMES.index(class_name)
        confidence = predictions[model_name][class_idx]
        row.append(confidence)
    confidence_matrix.append(row)
conf_df = pd.DataFrame(confidence_matrix, index=CLASS_NAMES, columns=list(model_paths.keys()))
print(conf_df.round(4))
print("="*70)
total_accuracy = sum(acc_dic.values())
weighted_confidences = {}
for class_name in CLASS_NAMES:
    class_idx = CLASS_NAMES.index(class_name)
    weighted_sum = 0
    for model_name in model_paths.keys():
        model_confidence = predictions[model_name][class_idx]
        model_weight = acc_dic[model_name] / total_accuracy
        weighted_sum += model_confidence * model_weight
    weighted_confidences[class_name] = weighted_sum
print("\n" + "="*70)
print("WEIGHTED CONFIDENCE FOR EACH CLASS")
print("-"*70)
for class_name, conf in weighted_confidences.items():
    print(f"{class_name:12s}: {conf:.4f} ({conf:.2%})")
print("-"*70)
final_class = max(weighted_confidences, key=weighted_confidences.get)
final_confidence = weighted_confidences[final_class]
print("\n" + "-"*70)
print("FINAL ENSEMBLE PREDICTION")
print("="*70)
print(f"Predicted Class: {final_class}")
print(f"Ensemble Confidence: {final_confidence:.4f} ({final_confidence:.2%})")




CONFIDENCE MATRIX (Class × Model)
          densenet121  efficientnetb3  resnet  vgg19
Immature        0.001             0.0     0.0    0.0
Mature          0.000             0.0     0.0    0.0
Normal          0.999             1.0     1.0    1.0

WEIGHTED CONFIDENCE FOR EACH CLASS
----------------------------------------------------------------------
Immature    : 0.0002 (0.02%)
Mature      : 0.0000 (0.00%)
Normal      : 0.9998 (99.98%)
----------------------------------------------------------------------

----------------------------------------------------------------------
FINAL ENSEMBLE PREDICTION
Predicted Class: Normal
Ensemble Confidence: 0.9998 (99.98%)


#*```CHECK FOR MODEL DISAGREEMENTS```*

In [44]:
individual_preds = {name: CLASS_NAMES[np.argmax(predictions[name])] for name in model_paths.keys()}
if len(set(individual_preds.values())) > 1:
    print(f"MODEL DISAGREEMENT DETECTED:")
    for model, pred in individual_preds.items():
        print(f"  {model:20s}: {pred}")
    print("Resolved using weighted confidence matrix approach")
else:
    print(f"All models agree on: {final_class}")
print("="*70)

All models agree on: Normal
